In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        

class DecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.class_to_idx = {c: idx for idx, c in enumerate(self.classes_)}
        self.n_classes_ = len(self.classes_)
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)
        return self

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == c) for c in self.classes_]
        predicted_class = self.classes_[np.argmax(num_samples_per_class)]
        node = Node(value=predicted_class)

        if (self.max_depth is None or depth < self.max_depth) and len(y) >= self.min_samples_split:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                if len(y_left) > 0 and len(y_right) > 0:
                    node.feature = idx
                    node.threshold = thr
                    node.left = self._grow_tree(X_left, y_left, depth + 1)
                    node.right = self._grow_tree(X_right, y_right, depth + 1)

        return node

    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None

        num_parent = [np.sum(y == c) for c in self.classes_]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None

        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent[:]
            for i in range(1, m):
                c = classes[i - 1]
                c_idx = self.class_to_idx[c]
                num_left[c_idx] += 1
                num_right[c_idx] -= 1
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes_))
                gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_))
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2

        return best_idx, best_thr

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        node = self.tree_
        path = []
        while node.feature is not None:
            if x[node.feature] < node.threshold:
                path.append(f"Feature {node.feature} < {node.threshold:.4f}")
                node = node.left
            else:
                path.append(f"Feature {node.feature} >= {node.threshold:.4f}")
                node = node.right
        if path:
            print("Splitting criteria for this prediction:")
            for crit in path:
                print(crit)
        return node.value